In [1]:
import numpy as np
from matplotlib import pyplot as plt
import py_gauge_mc
from tqdm import tqdm

In [2]:
replicas = 64
staging = 8
v_n = 64
L = 8
lf = py_gauge_mc.WindingNumberLeapfrog(replicas + staging, v_n, L,L,L,L)
lf.set_use_heatbath(True)

potentials_form = np.array([x**2 for x in range(0,v_n)], dtype=np.float32)

num_steps = 4
inv_ks = np.linspace(0.1,1.0,num_steps*(replicas - 1) + 1, dtype=np.float32)

# Lets check some seeding statistics

In [3]:
i = 0
sub_inv_ks = inv_ks[i*(replicas - 1):i*(replicas - 1) + replicas]
potentials = np.einsum('i,j->ij', sub_inv_ks, potentials_form)

def foo(x):
    y = [0,0,0,0,0,0]
    y[x] = 1
    return y
ws = [[0,0,0,0,0,0]] * 10
ws = ws + [foo(i) for i in range(6)]

lf.init_potentials(potentials, windings=np.array(ws, dtype=np.int32),num_staging=staging)

In [4]:
lf.repeated_seed(100, 10, 100, 10, 10, 10, True)

In [5]:
measurements = lf.repeated_seed_and_measure(100, 10, 100, 10, 10, 10, True)

In [14]:
states, energies = measurements

states

array([[[-1, -1,  1, -2,  1,  2],
        [ 3, -2, -2,  0,  2, -1],
        [ 2,  0, -2, -5, -2, -4],
        ...,
        [-1,  0, -2, -1,  0,  0],
        [ 0, -1,  0, -1,  0,  0],
        [-1,  2,  0,  0,  0,  1]],

       [[-1, -1,  0, -2,  0,  0],
        [-1, -1,  0,  0, -3,  0],
        [ 0,  1, -1, -1,  1, -1],
        ...,
        [ 1,  2,  0, -2,  0,  2],
        [ 2,  1,  0, -1,  1,  1],
        [ 0,  1, -1,  0,  1,  1]],

       [[-4,  2, -3, -5,  3, -4],
        [ 0,  1,  2, -2, -5, -2],
        [ 1,  0,  1,  0,  0,  0],
        ...,
        [ 0, -1,  1, -3, -1,  0],
        [-1, -1, -2,  1, -1, -1],
        [-1, -2,  0,  0,  0,  0]],

       ...,

       [[ 0, -1, -1,  0,  2,  0],
        [-3,  3, -1, -1, -1,  2],
        [ 1, -2,  0,  2, -1,  3],
        ...,
        [ 0,  0,  0, -1, -1,  1],
        [ 1,  0,  0,  1, -1,  0],
        [-1,  1, -1,  3,  0,  1]],

       [[-4, -3, -3,  0, -4,  2],
        [ 3,  0,  0, -8,  4, -1],
        [ 2,  1, -1, -2, -1, -1],
        .

# Time trials

In [3]:
# Time trial for seeding updates for i=0
i = 0
sub_inv_ks = inv_ks[i*(replicas - 1):i*(replicas - 1) + replicas]
potentials = np.einsum('i,j->ij', sub_inv_ks, potentials_form)
lf.init_potentials(potentials, windings=np.array([[0,0,0,0,0,0],[1,0,0,0,0,0]], dtype=np.int32),num_staging=staging)

def run():
    lf.repeated_seed(1)
    lf.wait_for_gpu()

%timeit run()

313 ms ± 8.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def run():
    lf.seed_random_winding(True)
    lf.wait_for_gpu()

%timeit run()

90.7 µs ± 8.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
def run():
    lf.run_global_sweep()
    lf.wait_for_gpu()

%timeit run()

105 µs ± 874 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
def run():
    lf.simulate_local(1)
    lf.wait_for_gpu()

%timeit run()

660 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
def run():
    lf.simulate_local(10)
    lf.wait_for_gpu()

%timeit run()

5.88 ms ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
def run():
    lf.run_parallel_tempering(False)
    lf.wait_for_gpu()

%timeit run()

3.88 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
def run():
    lf.run_parallel_tempering(True)
    lf.wait_for_gpu()

%timeit run()

3.92 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
def run():
    lf.get_energies()
    lf.wait_for_gpu()

%timeit run()

2.19 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
def run():
    lf.get_windings()
    lf.wait_for_gpu()

%timeit run()

2.13 ms ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
all_inv_ks = []
all_ws = []
all_es = []
sample_states = []

windings = None
counts = None
for i in tqdm(range(num_steps)):
    sub_inv_ks = inv_ks[i*(replicas - 1):i*(replicas - 1) + replicas]
    potentials = np.einsum('i,j->ij', sub_inv_ks, potentials_form)
    lf.init_potentials(potentials, windings=windings, standarize=True, num_staging=staging)
    lf.repeated_seed(10,
                    full_seed_steps_per_sample=2,
                    local_updates_after_seeding=100,
                    updates_between_seeding=32,
                    local_updates_before_tempering=5,
                    local_updates_after_tempering=5,
                    allow_inverting=True)
    w, e = lf.repeated_seed_and_measure(100,
                    full_seed_steps_per_sample=2,
                    local_updates_after_seeding=100,
                    updates_between_seeding=32,
                    local_updates_before_tempering=5,
                    local_updates_after_tempering=5,
                    allow_inverting=True)
    sample_states.append(lf.get_state_and_staging())
    windings = w[-1,:,:]
    all_inv_ks.append(sub_inv_ks)
    all_ws.append(w)
    all_es.append(e)
all_es = np.array(all_es)/L**4
all_ws = np.array(all_ws)
all_inv_ks = np.array(all_inv_ks)

  0%|                                                                                                                  | 0/4 [00:00<?, ?it/s]

In [ ]:
for ik, e in zip(all_inv_ks, all_es):
    plt.errorbar(ik, e.mean(axis=0), yerr=e.std(axis=0)/np.sqrt(e.shape[0]))
plt.grid()
plt.show()

In [ ]:
for ik, e in zip(all_inv_ks, all_es):
    plt.plot(ik, e.var(axis=0))
plt.grid()
plt.show()

In [ ]:
for ik, w in zip(all_inv_ks, all_ws):
    plt.plot(ik, w.var(axis=(-1,-3)))
plt.grid()
plt.show()

## Check old stuff...?

In [ ]:
old_all_inv_ks = []
old_all_ws = []
old_all_es = []

for i in tqdm(range(num_steps)):
    sub_inv_ks = inv_ks[i*(replicas - 1):i*(replicas - 1) + replicas]
    potentials = np.einsum('i,j->ij', sub_inv_ks, potentials_form)
    graph = py_gauge_mc.GPUGaugeTheory((L,L,L,L),potentials)
    graph.run_local_update(100)
    w, e = graph.simulate_and_get_winding_nums_and_energies(100, steps_per_sample=10, 
                                                            run_global_updates=True)
    old_all_inv_ks.append(sub_inv_ks)
    old_all_ws.append(w)
    old_all_es.append(e)
old_all_es = np.array(old_all_es)
old_all_ws = np.array(old_all_ws)

In [ ]:
for ik, e in zip(old_all_inv_ks, old_all_es):
    plt.errorbar(ik, e.mean(axis=-1), yerr=e.std(axis=-1)/np.sqrt(e.shape[-1]))
plt.grid()
plt.show()

In [ ]:
for ik, w in zip(old_all_inv_ks, old_all_ws):
    plt.plot(ik, w.var(axis=(-1,-2)))
plt.grid()
plt.show()